In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.optim as optim
import os 
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter


In [ ]:
ar1= np.load("/kaggle/input/tempnigga/output.npy")
print(ar1[0][:5])
ar= np.load("/kaggle/input/tempnigga/scales_smooth.npy")
print(ar[0][:5])

In [ ]:
class ResidualModel(nn.Module):
    def __init__(self):
        super(ResidualModel, self).__init__()
        self.fc0 = nn.Linear(24, 48)
        self.fc1 = nn.Linear(48, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 256)
        self.fc4 = nn.Linear(256, 283)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc0(x))
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [ ]:
class ResidualDataset(Dataset):
    def __init__(self, inputs , outputs):
        self.inputs = torch.FloatTensor(inputs)
        self.outputs = torch.FloatTensor(outputs)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

In [ ]:
inputs = np.load("/kaggle/input/tempnigga/invecs.npy")
outputs = np.load('/kaggle/input/tempnigga/outscales.npy')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResidualModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2) 
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3 )

# optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3)


train_dataset = ResidualDataset(inputs[:640], outputs[:640])
val_dataset = ResidualDataset(inputs[651:671], outputs[651:671])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True , num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False  , num_workers=4)


num_epochs = 261
start_epoch = 0
best_val_loss = float('inf')
patience = 15
no_improve = 0
count=0

print("Training started")
for epoch in range(start_epoch, num_epochs):
    model.train()
    train_loss = 0.0
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outs = model(inputs)
        loss = criterion(outs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() 
    
    train_loss /= len(train_dataset)
      

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outs = model(inputs)

            loss = criterion(outs, targets)
            val_loss += loss.item() 
    
    val_loss /= len(val_dataset)
    

    scheduler.step(val_loss)
    
#     if no_improve==7 and count==0 and optimizer.param_groups[0]['lr'] > 1e-6:
#         for param_group in optimizer.param_groups:
#             param_group['lr'] = 1e-9
#             print(f"Learning rate manually set to 5e-7 at epoch {epoch}")
#         count+=1

        
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
    if val_loss < best_val_loss :
        best_val_loss = val_loss
        no_improve = 0
        model_filename = f"epoch{epoch+1}.pth"
        model_path = os.path.join(".", model_filename)
        
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': train_loss,
        }
        torch.save(checkpoint, model_path)        
        print(f"Model saved at epoch {epoch+1}")
        
    else:
        no_improve += 1
        if no_improve == patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

print("Training completed")

In [ ]:
print(targets.shape)
print(optimizer.param_groups[0]["lr"])

In [ ]:
! rm -rf "/kaggle/working/"

In [ ]:
print(os.listdir("/kaggle/working"))

In [ ]:
outputs = np.load("/kaggle/input/tempnigga/a24_norm.npy")

labels = pd.read_csv("/kaggle/input/tempnigga/train_labels.csv")
labels = np.array(labels)


vals    = np.load('/kaggle/input/tempnigga/outputs.npy')
offsets = np.load('/kaggle/input/tempnigga/outscales.npy')

index = 671
inp = outputs[index]
value = vals[index]
true = labels[index][1:]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResidualModel().to(device)

weights="/kaggle/working/epoch11.pth"

print(inp.shape)
print(value.shape)
print(true.shape)
if weights:
    checkpoint = torch.load(weights, map_location=device , weights_only=True)
    model.load_state_dict(checkpoint['model_state_dict'])

model.eval()


with torch.no_grad():
    pred_scales = model(torch.tensor(inp).float())


# result = pred_scales[0] * value
result =  value /  pred_scales[0]



print(criterion(torch.tensor(result).unsqueeze(0) , torch.tensor(true).unsqueeze(0)))

# result = savgol_filter(result , 15 , 1)

# print(criterion(torch.tensor(result).unsqueeze(0) , torch.tensor(true).unsqueeze(0)))
print(criterion(torch.tensor(value).unsqueeze(0) , torch.tensor(true).unsqueeze(0)))




In [ ]:
torch.set_printoptions(precision=10)
print(type(pred_scales))

In [ ]:
print(result[:5])
print(value[:5])
print(pred_scales[0][:5])

print()
print(true[:5])



In [ ]:
tensor([0.0009970770, 0.0010203157, 0.0010123271, 0.0010105158, 0.0010154062])
[0.00165169 0.00176206 0.00177774 0.00163459 0.00168567]
tensor([1.6565283537, 1.7269738913, 1.7560904026, 1.6175847054, 1.6600964069])

[0.00108574 0.00113749 0.00113136 0.00112386 0.00113836]
